In [27]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Read initial data
file_path='rtfDataSet.csv'
df = pd.read_csv(file_path)
print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0 -0.536  0.292 -0.684  0.123 -0.118  0.346 -0.308 -0.113  0.401 -0.399  ...   
1 -0.496 -0.298 -0.529  0.073 -0.072  0.107 -0.316 -0.066 -0.168 -0.099  ...   
2  0.008 -0.031 -0.240  0.178 -0.120  0.317 -0.481  0.031 -0.077  0.063  ...   
3 -0.188 -0.180 -0.062 -0.104 -0.136 -0.061 -0.216 -0.143  0.068 -0.189  ...   
4  0.038  0.155 -0.203 -0.088 -0.084 -0.164 -0.145 -0.168  0.008  0.143  ...   

      61     62     63     64     65     66     67     68     69  label  
0 -0.135 -0.063 -0.410 -0.223 -0.599 -0.136 -0.329 -0.132 -0.266      0  
1  0.189  0.007 -0.362 -0.151 -0.338 -0.031 -0.159 -0.097 -0.131      0  
2 -0.035  0.101 -0.098 -0.172 -0.290  0.033 -0.342 -0.321 -0.238      0  
3  0.146 -0.145 -0.527 -0.292 -0.663 -0.078 -0.194 -0.151 -0.268      0  
4  0.019 -0.084  0.164 -0.165 -0.126 -0.112  0.029 -0.138  0.148      1  

[5 rows x 71 columns]


In [19]:
#Reduce to sets and normalized
set_1=[1, 5, 10, 13, 14, 16, 17, 19, 25, 26, 29, 30, 36, 46, 51, 57]
set_2=[2, 5, 9, 10, 16, 21, 28, 34, 38, 48, 49, 55, 61, 62, 64]
set_3=[1, 2, 7, 8, 9, 10, 19, 23, 28, 34, 35, 36, 42, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 63, 64, 65, 67]
set_4=[2, 3, 12, 16, 21, 23, 28, 34, 35, 45, 48, 50, 53, 61]
set_5=[2, 10, 11, 12, 14, 22, 36, 41, 46, 50, 55, 56, 57, 58, 65]

reduced_df_1=df.iloc[:,set_1]
reduced_df_2=df.iloc[:,set_2]
reduced_df_3=df.iloc[:,set_3]
reduced_df_4=df.iloc[:,set_4]
reduced_df_5=df.iloc[:,set_5]
y = df.iloc[:, -1]

scaler = MinMaxScaler()
reduced_df_1 = scaler.fit_transform(reduced_df_1)
reduced_df_2 = scaler.fit_transform(reduced_df_2)
reduced_df_3 = scaler.fit_transform(reduced_df_3)
reduced_df_4 = scaler.fit_transform(reduced_df_4)
reduced_df_5 = scaler.fit_transform(reduced_df_5)

In [20]:
#Array of sets
reduced_array=[reduced_df_1,reduced_df_2,reduced_df_3,reduced_df_4,reduced_df_5]

In [26]:
#Creacion dinamica de folds y clasificacion y metricas
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter=1
for X in reduced_array:
    print("Set: ",counter)
    mean_accuracy=0
    mean_precision=0
    mean_recall=0
    for fold,(train_index,val_index) in enumerate(stratified_kfold.split(X,y)):
        X_train,X_val=X[train_index],X[val_index]
        y_train,y_val=y[train_index],y[val_index]

        clf = GaussianNB()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        #auc=roc_auc_score(y_val, y_pred) Esto no sirve falta AUC/ROC
        mean_accuracy+=accuracy
        mean_precision+=precision
        mean_recall+=recall

    mean_accuracy=mean_accuracy/10
    mean_precision=mean_precision/10
    mean_recall=mean_recall/10
    print("Mean precision: ",mean_precision)
    print("Mean recall: ",mean_recall)
    print("Mean accuracy: ",mean_accuracy)
    counter+=1


Dataset:  1
Fold 1 - Precisión: 0.725
Fold 2 - Precisión: 0.55
Fold 3 - Precisión: 0.625
Fold 4 - Precisión: 0.675
Fold 5 - Precisión: 0.7
Fold 6 - Precisión: 0.7
Fold 7 - Precisión: 0.6923076923076923
Fold 8 - Precisión: 0.5897435897435898
Fold 9 - Precisión: 0.6410256410256411
Fold 10 - Precisión: 0.6410256410256411
Dataset:  2
Fold 1 - Precisión: 0.775
Fold 2 - Precisión: 0.65
Fold 3 - Precisión: 0.6
Fold 4 - Precisión: 0.775
Fold 5 - Precisión: 0.7
Fold 6 - Precisión: 0.675
Fold 7 - Precisión: 0.6666666666666666
Fold 8 - Precisión: 0.7692307692307693
Fold 9 - Precisión: 0.5384615384615384
Fold 10 - Precisión: 0.6923076923076923
Dataset:  3
Fold 1 - Precisión: 0.8
Fold 2 - Precisión: 0.65
Fold 3 - Precisión: 0.65
Fold 4 - Precisión: 0.8
Fold 5 - Precisión: 0.725
Fold 6 - Precisión: 0.65
Fold 7 - Precisión: 0.717948717948718
Fold 8 - Precisión: 0.6410256410256411
Fold 9 - Precisión: 0.5897435897435898
Fold 10 - Precisión: 0.717948717948718
Dataset:  4
Fold 1 - Precisión: 0.825
Fold 2